In [ ]:
# 목록 저장. 매일신문은 시간을 좀 오래 기다려야 뜨는 일이 많음.

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import pandas as pd
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# 크롬 드라이버 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")  # 필요시 활성화
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--blink-settings=imagesEnabled=false")  # 이미지 로드 비활성화
chrome_options.add_argument("--disable-extensions")  # 확장 프로그램 비활성화
chrome_options.add_argument("--disable-gpu")  # GPU 가속 비활성화

# 크롬 드라이버 설정
driver = webdriver.Chrome(options=chrome_options)
driver.maximize_window()
wait = WebDriverWait(driver, 10)

# 키워드 목록과 날짜 범위 설정
keywords = [
    "검색어",
]
start_date = "2024-06-01"
end_date = "2024-08-31"

collected_urls = set()
all_articles = []

def search_and_save_articles(keyword):
    url = "https://search.imaeil.com/RSA/front_new/Search.jsp"
    driver.get(url)
    time.sleep(2)

    # 검색어 입력
    search_box = driver.find_element(By.ID, 'qt')
    search_box.send_keys(keyword)
    search_box.submit()

    try:
        more_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//a[contains(@title, "더보기")]')))
        more_button.click()
        print(f"'더보기' 버튼 클릭 완료.")
    except Exception as e:
        print(f"'더보기' 버튼 클릭 실패 또는 존재하지 않음: {e}")

    articles = []

    def extract_articles():
        try:
            wait.until(EC.presence_of_element_located((By.ID, 'board_sc')))
            time.sleep(5)  # 'board_sc' 요소가 나타날 때까지 대기
        except Exception as e:
            print(f"Error waiting for articles section: {e}")
            return False
    
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        articles_section = soup.find("div", id="board_sc")  # 기사 섹션 확인
        time.sleep(5)
        
        # 기사 섹션이 제대로 있는지 확인
        if not articles_section:
            print("No articles section found!")
            return False

        articles_list = articles_section.find_all("li", class_="txt")
        if not articles_list:
            print("No articles found!")
            return False

        # 기사 정보 추출
        for item in articles_list:
            try:
                # 제목 및 링크 추출
                title_tag = item.find("a", title=True)
                if title_tag:
                    title = title_tag["title"].strip()
                    url = title_tag["href"]
                else:
                    title = ""
                    url = ""

                # 날짜는 해당 아이템의 다음 li.txt2에서 추출
                date_element = item.find_next_sibling("li", class_="txt2").find("span", class_="wGun")
                date = date_element.text.strip().split()[0] if date_element else ""

                # 시작 날짜 이전 기사면 종료
                if date < start_date:
                    print(f"기사가 시작 날짜 이전입니다. 스크래핑을 중단합니다: {date}")
                    return False  # 루프 종료

                # 종료 날짜 이후의 기사는 무시
                if date > end_date:
                    continue  # 범위 밖의 기사는 스킵

                if url in collected_urls:
                    continue
                collected_urls.add(url)

                # 기사 데이터 추가
                articles.append({
                    'media': '매일신문',
                    'title': title,
                    'date': date,
                    'section': '',
                    'article_type': '',
                    'writer': '',
                    'page': '',
                    'url': url,
                    'content': '' 
                })
                print(f"Added article: {title} | {date} | {url}")
            except Exception as e:
                print(f"Error processing article: {e}")
        return True

    def scrape_page_group():
        page_num = 1
        while True:
            try:
                if page_num == 1:
                    # 첫 페이지는 이미 로드되었으므로 추가 클릭이 필요 없음
                    pass
                else:
                    page_link = wait.until(EC.element_to_be_clickable((By.XPATH, f'//a[text()="{page_num}"]')))
                    page_link.click()

                if not extract_articles():
                    break  # 날짜 조건에 의해 중단

                # >
                if page_num % 10 == 0:  # Click ">" button after every 10 pages
                    try:
                        next_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//a[contains(@onclick, "clickFldSet") and contains(text(), ">")]')))
                        next_button.click()
                        time.sleep(2)  # 필요 시 조정
                        page_num = 0  # 다음 그룹의 페이지 번호 초기화
                    except Exception as e:
                        print(f"'>' 버튼 클릭 실패: {e}")
                        break

                page_num += 1
            
            except Exception as e:
                print(f"Page navigation error: {e}")
                break
        
        return articles
    
    scrape_page_group()

    print(f"키워드 '{keyword}'에 대해 수집된 기사 수: {len(articles)}")

    return articles

# 모든 키워드에 대해 함수 호출
for keyword in keywords:
    print(f"\n키워드 검색 시작: {keyword}")
    articles = search_and_save_articles(keyword)
    if not articles:
        print(f"키워드 '{keyword}'에 대한 기사가 더 없습니다. 다음 키워드로 이동합니다.")
        continue  # 다음 키워드로 이동

    all_articles.extend(articles)

# 기사 추출 함수 호출

    # 데이터프레임으로 변환
    df = pd.DataFrame(all_articles)
    df = df[~df['title'].str.contains('블락비|나인뮤지스', case=False, na=False)]        
    df.to_excel(f'2매일신문 {start_date}~{end_date}.xlsx', index=False)
    time.sleep(2)  # 각 키워드 사이에 잠시 대기    


driver.quit()

In [ ]:
# 본문 추출

import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup

# Selenium 헤드리스 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# 1. 출처와 파일명 변수 설정
file_name = '파일명'  # 파일명 설정
base_path = rf'경로'  # 경로 설정

# 2. 임시 파일 경로와 최종 파일 경로 생성
temp_output_path = os.path.join(base_path, f'{file_name}temp.xlsx')
output_path = os.path.join(base_path, f'{file_name}_본문.xlsx')  # 최종 파일 경로
error_log_path = os.path.join(base_path, f'{file_name}_error_log.txt')  # 에러 로그 파일 경로

# 3. 파일 경로 생성 및 엑셀 파일 불러오기
input_file = os.path.join(base_path, f'{file_name}.xlsx')
df = pd.read_excel(input_file, engine='openpyxl')

# ChromeDriver 설정 없이 크롬 브라우저 실행
driver = webdriver.Chrome(options=chrome_options)

def wait_for_article_body(wait, class_names):
    """주어진 클래스 이름 중 첫 번째로 매칭된 요소를 반환합니다."""
    for class_name in class_names:
        try:
            return wait.until(EC.presence_of_element_located((By.CLASS_NAME, class_name)))
        except TimeoutException:
            continue 
    return None

def get_content_and_subtitle_from_url(url):
    try:
        driver.get(url)

        # WebDriverWait 설정
        wait = WebDriverWait(driver, 5)

        # 기사 본문을 찾기 위한 클래스 이름 리스트
        class_names = ['article_content', 'articlebody', 'newsct_article', 'article-txt', 'story-news', 'content01']

        # 첫 번째로 매칭된 요소 찾기
        article_body_element = wait_for_article_body(wait, class_names)

        if article_body_element:
            # 페이지 소스에서 BeautifulSoup으로 변환
            soup = BeautifulSoup(driver.page_source, 'html.parser')

            # 서브 타이틀 추출 및 제거
            subtitle = ""
            subtitle_tag = soup.find('p', class_='subtitle')
            if subtitle_tag:
                subtitle = subtitle_tag.get_text(separator='\n', strip=True)
                subtitle_tag.decompose()  # 서브 타이틀 제거

            # 'figure', 'table', 'div' 태그 및 기타 불필요한 태그 제거
            for tag in soup.find_all(['figure', 'table', 'div'], class_=['img_center', 'article-figure', 'nbd_table', 'comp-box photo-group', 'article-img', 'img-con', 'chn_box']):
                tag.decompose()

            # 본문에서 <span data-type="ore"> 태그를 제거하고 안의 텍스트만 남김
            for span_tag in soup.find_all('span', {'data-type': 'ore'}):
                span_tag.unwrap()  # <span> 태그를 제거하고 텍스트만 유지

            # 본문 내용 추출
            article_body = soup.find(class_=class_names)
            if article_body:
                content = article_body.get_text(separator='\n', strip=True)
            else:
                print(f"본문을 찾지 못했습니다: {url}")
                content = ""

            # 저작권 정보 제거
            txtcopyright = soup.find('p', class_='txt-copyright')
            if txtcopyright:
                txtcopyright.decompose()

            return content, subtitle
        else:
            return "", ""

    except Exception as e:
        print(f"Error fetching URL {url}: {e}")
        return None, None

# 진행 중인 기사 스크래핑 상태를 트래킹할 리스트와 로그 파일
error_log = []
total_articles = len(df)

# 각 행의 URL에서 본문과 서브 타이틀 내용을 스크랩하여 content와 sub-title 컬럼에 저장합니다.
for index, row in df.iterrows():
    print(f"Processing {index + 1}/{total_articles}: {row['url']}")
    content, subtitle = get_content_and_subtitle_from_url(row['url'])
    if content is None:
        error_log.append(row['url'])
    df.at[index, 'content'] = content
    df.at[index, 'sub-title'] = subtitle  # 서브 타이틀 저장

    # 100개마다 파일 저장
    if (index + 1) % 50 == 0:
        df.to_excel(temp_output_path, index=False)
        print(f"임시 파일을 저장했습니다: {temp_output_path}")

# 웹드라이버 종료
driver.quit()

# 에러 로그 파일 저장
if error_log:
    with open(error_log_path, 'w') as f:
        for url in error_log:
            f.write(f"{url}\n")
    print(f"에러 로그가 저장되었습니다: {error_log_path}")

# 변경된 데이터를 파일로 저장합니다.
df = df[~df['title'].str.contains('블락비|나인뮤지스', case=False, na=False)]
df = df[~df['content'].str.contains('블락비|나인뮤지스', case=False, na=False)]
df.to_excel(output_path, index=False)
print("본문 내용을 성공적으로 스크랩하여 저장했습니다.")
